In [ ]:
######APP install ADS ANALYSIS
import os
import pandas as pd
import numpy as np

path = '/Users/kumaku03/Desktop/creatives/Image/app_install/'
####Getting all the app Install ads campaign metrics from Google Ads
df1 = pd.read_excel(path + "app_install_data.xlsx") 
df1.head()

df1.shape

####Getting only the first Image ads from the record
cols = ['Image 1', 'CTR', 'Conversions', 'Impressions', 'Clicks', 'Cost' ]
df2 = df1[cols]


df2 = df2.dropna()
import re
df2['Image_p'] = df2["Image 1"].apply(lambda x: (re.sub("images/3723194194/", '',x)))

df2.head()

####GETTING THE MAXIMUM CTR for each Image
idx = df2.groupby(['Image_p'])['CTR'].transform(max) == df2['CTR']
aa = df2[idx]

idx = aa.groupby(['Image_p'])['Impressions'].transform(max) == aa['Impressions']
df3 = aa[idx]
df3 = df3.reset_index(drop = True)

#####Loading the Individual App Install Ads extraction metrics from text extraction, NLP code

path = '/Users/kumaku03/Desktop/creatives/img_text/'

df4 = pd.read_excel(path + "image_data_text_app.xlsx")

df4.head()

df5 = pd.merge(df3, df4, left_on='Image_p', right_on='Image')

#cols = ['Campaign', 'Ad Group', 'Image 1', 'Image 2', 'Image 3', 'Image 4' ]
#X = ddf[cols]

#df_f = pd.melt(X, ["Campaign", "Ad Group"], var_name="Image", value_name="Path")
#df_f.dropna(axis=0)

ddf[['CTR']].boxplot()

####Get the 90th and 10th percentile values
q3, q1 = np.percentile(ddf['CTR'], [90 ,10])
q3, q1

q3, q1 = np.percentile(ddf['CTR'], [75 ,25])
q3, q1


#####LINEAR REGRESSION

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

feature_cols = [
'text_perc',
'person_perc',
'contrast_r',
#'height',
#'width',
'word_count',
'avg_word_length',  
#'char_count',
'person_count',
#'sentiment'
    
#'Clicks',
#'Cost',
#'Impressions',
#'CTR',
#'Avg CPC',
#'Avg CPM',
#'CTR',    
]
y_cols = ['CTR']                
#y_cols = ['Conversions']

X = ddf[feature_cols] # Features
y = ddf[y_cols] # Target variable

y = y*100

#####MULTI COLLINEARITY MATRIX, Calculating the correlation between the variables
xx = X.corr()

import matplotlib.pyplot as plt
import seaborn as sns
# Visualize this transition matrix in a heatmap
sns.set_style("whitegrid")
fig, ax = plt.subplots(figsize=(12, 6))
sns.heatmap(xx, cmap="RdBu_r",annot=True,linewidths=.5,)
plt.show()

####CALCULATING the VIF of the variables

vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
from statsmodels.stats.outliers_influence import variance_inflation_factor
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
vif_data

# now lets split the data into train and test
from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state = 200 )
X_train.shape, X_test.shape, y_train.shape, y_test.shape


xx = X.corr()

import matplotlib.pyplot as plt
import seaborn as sns


# Visualize this transition matrix in a heatmap
sns.set_style("whitegrid")
fig, ax = plt.subplots(figsize=(12, 6))
sns.heatmap(xx, cmap="RdBu_r",annot=True,linewidths=.5,)
plt.show()


#####RANDOM FOREST REGRESSOR MODEL

from sklearn.ensemble import RandomForestRegressor
#create an object of the DecisionTree class.
rf_regressor = RandomForestRegressor(n_estimators = 100, random_state = 100, max_depth=3)

#fit it on the data, do not need to fit_transform her
rf_regressor.fit(X_train, y_train)

####XG Boosting #########
import xgboost as xg

xgb_r = xg.XGBRegressor(objective ='reg:linear', n_estimators = 100, seed = 123)

####Fitting the train dataset
xgb_r.fit(X_train, y_train) 

# Predict the model for TEST dataset
y_pred = xgb_r.predict(X_test)

rom sklearn.metrics import mean_squared_error
MSE = mean_squared_error(y_test, y_pred)
MSE

RMSE = np.sqrt(MSE)
RMSE
print("XG Boosting - RMSE value for Test dataset: " + str(round(RMSE,3)))

#FEATURE IMPORTANCE 
import pandas as pd
feature_imp = pd.Series(xgb_r.feature_importances_,index=feature_cols).sort_values(ascending=False)
feature_imp






